In [ ]:
from __future__ import annotations

from pathlib import Path
from typing import Union, Dict, Any
import random
import numpy as np

from ultralytics import YOLO
import torch


def set_seed(seed: int = 42) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


def train_yolo(
    data: Union[str, Path, Dict[str, Any]],
    weights: Union[str, Path] = "yolo11n-seg.pt",
    *,
    epochs: int = 50,
    imgsz: int = 640,
    batch: int = -1,
    device: str = "cpu",
    workers: int = 8,
    project: Union[str, Path] = "runs/train",
    name: str = "exp",
    seed: int = 42,
):

    set_seed(seed)
    model = YOLO(str(weights))
    args = dict(
        data=(str(data) if isinstance(data, (str, Path)) else data),
        epochs=epochs,
        imgsz=imgsz,
        batch=batch,
        device=device,
        workers=workers,
        project=str(project),
        name=name,
        seed=seed,
        val=True,
        plots=True,
        exist_ok=True,
        save=True,
    )
    results = model.train(**args)
    metrics = model.val(data=args["data"], imgsz=imgsz, device=device, split="val", plots=True)
    save_dir = Path(getattr(results, "save_dir", Path(project) / name))
    return {"save_dir": save_dir, "metrics": metrics}


In [ ]:

DATA = {
    "train": "/home/jupyter/mnt/datasets/real_data/images/train",
    "val":   "/home/jupyter/mnt/datasets/real_data/images/val",
    "names": ["10_Combination_Wrench_3_4", "11_Diagonal_Cutters", "1_Flathead_Screwdriver", "2_Phillips_Screwdriver", "3_Pozidriv_Screwdriver", "4_Hand_Drill", "5_Safety_Wire_Pliers", "6_Slip_Joint_Pliers", "7_Circlip_Pliers", "8_Adjustable_Wrench", "9_Oil_Can_Opener"]
}

WEIGHTS = "best.pt"

PARAMS = dict(
    epochs=50,
    imgsz=640,
    batch=32,
    device="0",
    workers=8,
    project="runs/train",
    name="exp_clean",
    seed=42,
)


In [ ]:
summary = train_yolo(DATA, WEIGHTS, **PARAMS)
summary
